In [1]:
import os
import csv
import glob
import random
import scipy.ndimage
import numpy as np
import tensorflow as tf
from imutils import paths
from keras.applications.inception_v3 import InceptionV3
from sklearn.preprocessing import LabelEncoder
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
ROOT_DIR = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath('__file__'))))
SOURCE_DIR = os.path.abspath('/data/aumkar/train')
TARGET_DIR = os.path.abspath('/data/aumkar/validation')
DATA_LOAD_DIR = os.path.abspath('/data/aumkar/data_load')

In [4]:
source_path = list(paths.list_images(SOURCE_DIR))
target_path = list(paths.list_images(TARGET_DIR))

In [5]:
random.shuffle(source_path)

In [6]:
random.shuffle(target_path)

In [7]:
labels_source = [p.split(os.path.sep)[-2] for p in source_path]
labels_target = [p.split(os.path.sep)[-2] for p in target_path]

In [8]:
le = LabelEncoder()
labels = le.fit_transform(labels_source)
labels_t = le.fit_transform(labels_target)

In [11]:
labels_unique = np.unique(labels)

In [12]:
labels_unique

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [13]:
np.save(os.path.join(DATA_LOAD_DIR, 'labels.npy'), labels_unique)

In [ ]:
def model_create(transfer_model):
    if transfer_model == 'InceptionV3':
        with tf.device('/gpu:1'):
            return InceptionV3(weights = 'imagenet', include_top = False)
    elif transfer_model == 'VGG16':
        with tf.device('/gpu:1'):
            return VGG16(weights = 'imagenet', include_top = False)
    elif transfer_model == 'VGG19':
        with tf.device('/gpu:1'):
            return VGG19(weights = 'imagenet', include_top = False)
    elif transfer_model == 'Xception':
        with tf.device('/gpu:1'):
            return Xception(weights = 'imagenet', include_top = False)
    elif transfer_model == 'DenseNet121':
        with tf.device('/gpu:1'):
            return DenseNet121(weights = 'imagenet', include_top = False)

In [ ]:
def model_reshape(transfer_model, features):
    if transfer_model == 'InceptionV3':
        return features.reshape((features.shape[0], 5 * 5 * 2048))
    elif transfer_model == 'VGG16':
        return features.reshape((features.shape[0], 7 * 7 * 512))
    elif transfer_model == 'VGG19':
        return features.reshape((features.shape[0], 7 * 7 * 512))
    elif transfer_model == 'Xception':
        return features.reshape((features.shape[0], 7 * 7 * 2048))
    elif transfer_model == 'DenseNet121':
        return features.reshape((features.shape[0], 7 * 7 * 1024))

In [ ]:
def model_save(transfer_model, features, label, b, source):
    if transfer_model == 'InceptionV3' and source == 'source':
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'inception', 'feat_incept_%s.npz' % b), a = features)
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'inception', 'feat_incept_label_%s.npz' % b), a = label)
    elif transfer_model == 'InceptionV3' and source == 'target':
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'inception', 'feat_incept_target_%s.npz' % b), a = features)
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'inception', 'feat_incept_label_target_%s.npz' % b), a = label)
    elif transfer_model == 'VGG16' and source == 'source':
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'vgg16', 'feat_vgg16_%s.npz' % b), a = features)
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'vgg16', 'feat_vgg16_label_%s.npz' % b), a = label)
    elif transfer_model == 'VGG16' and source == 'target':
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'vgg16', 'feat_vgg16_target_%s.npz' % b), a = features)
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'vgg16', 'feat_vgg16_label_target_%s.npz' % b), a = label)
    elif transfer_model == 'VGG19' and source == 'source':
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'vgg19', 'feat_vgg19_%s.npz' % b), a = features)
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'vgg19', 'feat_vgg19_label_%s.npz' % b), a = label)
    elif transfer_model == 'VGG19' and source == 'target':
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'vgg19', 'feat_vgg19_target_%s.npz' % b), a = features)
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'vgg19', 'feat_vgg19_label_target_%s.npz' % b), a = label)
    elif transfer_model == 'Xception' and source == 'source':
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'xception', 'feat_xcept_%s.npz' % b), a = features)
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'xception', 'feat_xcept_label_%s.npz' % b), a = label)
    elif transfer_model == 'Xception' and source == 'target':
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'xception', 'feat_xcept_target_%s.npz' % b), a = features)
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'xception', 'feat_xcept_label_target_%s.npz' % b), a = label)
    elif transfer_model == 'DenseNet121' and source == 'source':
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'densenet', 'feat_dense_%s.npz' % b), a = features)
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'densenet', 'feat_dense_label_%s.npz' % b), a = label)
    elif transfer_model == 'DenseNet121' and source == 'target':
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'densenet', 'feat_dense_target_%s.npz' % b), a = features)
        np.savez_compressed(os.path.join(DATA_LOAD_DIR, 'densenet', 'feat_dense_label_target_%s.npz' % b), a = label)

In [ ]:
def feature_extract(path, batch_size, labels, model_, source):
    
    model1 = model_create(model_)
    
    for (b, i) in enumerate(range(0, len(path), batch_size)):
        # extract the batch of images and labels, then initialize the
        # list of actual images that will be passed through the network
        # for feature extraction
        print("[INFO] processing batch {}/{}".format(b + 1, int(np.ceil(len(path) / float(batch_size)))))
        batchPaths = path[i: i + batch_size]
        batchLabels = labels[i: i + batch_size]
        batchImages = []

        # loop over the images and labels in the current batch
        for imagePath in batchPaths:
            # load the input image using the Keras helper utility while
            # ensuring the image is resized to 224x224 pixels
            image = load_img(imagePath, target_size=(224, 224))
            image = img_to_array(image)

            # preprocess the image by (1) expanding the dimensions and
            # (2) subtracting the mean RGB pixel intensity from the
            # ImageNet dataset
            image = np.expand_dims(image, axis=0)
            image = imagenet_utils.preprocess_input(image)

            # add the image to the batch
            batchImages.append(image)

        # pass the images through the network and use the outputs as our
        # actual features, then reshape the features into a flattened
        # volume
        batchImages = np.vstack(batchImages)
            
        features = model1.predict(batchImages, batch_size = batch_size)
        features1 = model_reshape(model_, features)

        model_save(model_, features1, batchLabels, b, source)

In [ ]:
feature_extract(source_path, 64, labels, 'InceptionV3', 'source')

In [ ]:
feature_extract(target_path, 64, labels_t, 'InceptionV3', 'target')